In [ ]:
import numpy as np
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD


##Loading Data

Each datapoint is a 32x32 pixel color photographs

Classes : 10

[0: airplane]
[1: automobile]
[2: bird]
[3: cat]
[4: deer]
[5: dog]
[6: frog]
[7: horse]
[8: ship]
[9: truck]

Samples per class ~6000

Samples total : 60000

In [ ]:
#Scale Pixels
def prep_pixels(trainX, testX):
	# convert from integers to floats
	train_norm = trainX.astype('float32')
	test_norm = testX.astype('float32')
 
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	
	return train_norm, test_norm


# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
 
  # summarize loaded dataset
	print('Train: X=%s, y=%s' % (trainX.shape, trainY.shape))
	print('Test: X=%s, y=%s' % (testX.shape, testY.shape))
	
	# scale pixels
	trainX, testX = prep_pixels(trainX, testX)
 
	return trainX, trainY, testX, testY

##Shuffling Data

In [ ]:
def shuffle_dataset(trainX,trainY,testX,testY):
  # number of datapoints [0,1,2,.....,n]
  no_dataPoints_train = np.arange(0,trainX.shape[0]) # 50000
  no_dataPoints_test = np.arange(0,testX.shape[0])   # 10000

  # random shuffling
  np.random.shuffle(no_dataPoints_train)
  np.random.shuffle(no_dataPoints_test)

  # reshuffled original train data
  X_train = trainX[no_dataPoints_train]
  y_train = trainY[no_dataPoints_train]

  # reshuffled original test data
  X_test = testX[no_dataPoints_test]
  y_test = testY[no_dataPoints_test]

  return X_train,y_train,X_test,y_test

#Loading dataset and Storing label specific index


In [ ]:
trainX, trainY, testX, testY = load_dataset()
trainX, trainY, testX, testY = shuffle_dataset(trainX, trainY, testX, testY)

# each class datapoints index
each_Cls_DataIndex_train = []

for i in range(10) :
  _, class_i_train = np.where([trainY[:,0]==i]) # trainY = [[1],[2],[3]....]
  each_Cls_DataIndex_train.append(class_i_train)
  
# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)

170508288/170498071 [==============================] - 3s 0us/step
Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


##Central Data Preparation

In [ ]:
# 30% data samples to central 
n_pts_training = int((trainX.shape[0]*.3)/10)

print("Number of dataPoints from each class for training : ",n_pts_training)

print("Number of dataPoints for training : ",n_pts_training*10)

Number of dataPoints from each class for training :  1500
Number of dataPoints for training :  15000


In [ ]:
central_dataPoint_index_train = []

for i in range(10) :
  ind1 = each_Cls_DataIndex_train[i][0:n_pts_training]
  central_dataPoint_index_train.extend(ind1)  # extend so that 1D [ind[0], ind[1],...] array not like this [[ind]]

X_central_train = trainX[central_dataPoint_index_train]
y_central_train = trainY[central_dataPoint_index_train]


print("Central Model dataset")
print("X_train : ", X_central_train.shape, "y_train : ", y_central_train.shape)
print("X_validate : ", testX.shape, "y_validate : ", testY.shape)

Central Model dataset
X_train :  (15000, 32, 32, 3) y_train :  (15000, 10)
X_validate :  (10000, 32, 32, 3) y_validate :  (10000, 10)


##Central Model ( Basic Arch )

In [ ]:
# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(10, activation='softmax'))
  return model

In [ ]:
model = define_model()
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                    

In [ ]:
# compile model
model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_central_train, y_central_train, epochs=20, batch_size=100, validation_data=(testX, testY))

Epoch 1/20
150/150 [==============================] - 16s 39ms/step - loss: 2.0823 - accuracy: 0.2432 - val_loss: 1.9006 - val_accuracy: 0.3206
Epoch 2/20
150/150 [==============================] - 6s 42ms/step - loss: 1.7948 - accuracy: 0.3629 - val_loss: 1.7059 - val_accuracy: 0.3855
Epoch 3/20
150/150 [==============================] - 5s 35ms/step - loss: 1.6716 - accuracy: 0.4029 - val_loss: 1.6184 - val_accuracy: 0.4205
Epoch 4/20
150/150 [==============================] - 5s 35ms/step - loss: 1.5643 - accuracy: 0.4365 - val_loss: 1.5046 - val_accuracy: 0.4609
Epoch 5/20
150/150 [==============================] - 5s 34ms/step - loss: 1.4689 - accuracy: 0.4742 - val_loss: 1.4596 - val_accuracy: 0.4750
Epoch 6/20
150/150 [==============================] - 5s 36ms/step - loss: 1.4175 - accuracy: 0.4917 - val_loss: 1.3904 - val_accuracy: 0.5044
Epoch 7/20
150/150 [==============================] - 5s 34ms/step - loss: 1.3441 - accuracy: 0.5207 - val_loss: 1.3733 - val_accuracy: 0.505

In [ ]:
weights_central = model.get_weights()
model.save("model.h5")

In [ ]:
accuracy_central_Avg = []
accuracy_central_GMM = []

_, accuracy = model.evaluate(testX, testY,verbose=0) # returns loss and accuracy
accuracy_central_Avg.append(round(accuracy,2))
accuracy_central_GMM.append(round(accuracy,2))


##Data Preparation for Local Models

###Per local client ==> 3500 class specific point
creating a mixture dataPoints to add to all this ==> 900 (90 from each class)

In [ ]:
# making a mixture data

mix_Xy_train = []

for i in range(10):
  ind1 = each_Cls_DataIndex_train[i][n_pts_training:n_pts_training+900] # 90 from 10 classes ==> 900
  
  X_mix_train = trainX[ind1]
  y_mix_train = trainY[ind1]
  
  mix_Xy_train.append((X_mix_train,y_mix_train))

n_pts_training += 900

# without data change cycle 

In [ ]:
# # Control variable 
# onlyOnce = True
# onlyIter = True;


# local_Xy_train_batch = []

# # in each batch will mix 10 data points
# index_mix_train = [0,0,0,0,0,0,0,0,0,0]

# for j in range(10):
#   local_Xy_train = []

#   for i in range(1) :
#     ind1 = each_Cls_DataIndex_train[i][n_pts_training:n_pts_training+2600]
  
#     X_local_train = trainX[ind1]
#     y_local_train = trainY[ind1]
#     # print(len(X_local_train), len(y_local_train))
    
    
#     for k in range(10):
      
#       if(k!=i):
#         X_cat_b_t = np.concatenate((X_local_train , mix_Xy_train[k][0][index_mix_train[k]:index_mix_train[k]+10]),axis=0)
#         y_cat_b_t = np.concatenate((y_local_train , mix_Xy_train[k][1][index_mix_train[k]:index_mix_train[k]+10]),axis=0)
        
#         index_mix_train[k] += 10;

#         X_local_train = X_cat_b_t
#         y_local_train = y_cat_b_t
 
      
#     if(onlyOnce):
#       if(onlyIter):
#         print("Iteration : " , j)
#         onlyIter = False;
#       print("Local Clent ", i);
#       print("Training data X y : ", X_local_train.shape, y_local_train.shape)
    
#     local_Xy_train.append((X_cat_b_t,y_cat_b_t))
    
#   onlyOnce = False
#   local_Xy_train_batch.append(local_Xy_train)

Iteration :  0
Local Clent  0
Training data X y :  (2690, 32, 32, 3) (2690, 10)


#With data change

In [ ]:
# Control variable 
onlyOnce = True
onlyIter = True;


local_Xy_train_batch = []

# in each batch will mix 10 data points
index_mix_train = [0,0,0,0,0,0,0,0,0,0]

for j in range(10):
  local_Xy_train = []

  for i in range(10) :
    ind1 = each_Cls_DataIndex_train[i][n_pts_training:n_pts_training+260]
  
    X_local_train = trainX[ind1]
    y_local_train = trainY[ind1]
    # print(len(X_local_train), len(y_local_train))
    
    
    for k in range(10):
      
      if(k!=i):
        X_cat_b_t = np.concatenate((X_local_train , mix_Xy_train[k][0][index_mix_train[k]:index_mix_train[k]+90]),axis=0)
        y_cat_b_t = np.concatenate((y_local_train , mix_Xy_train[k][1][index_mix_train[k]:index_mix_train[k]+90]),axis=0)
        
        index_mix_train[k] += 90;

        X_local_train = X_cat_b_t
        y_local_train = y_cat_b_t
 
      
    if(onlyOnce):
      if(onlyIter):
        print("Iteration : " , j)
        onlyIter = False;
      print("Local Clent ", i);
      print("Training data X y : ", X_local_train.shape, y_local_train.shape)
    
    local_Xy_train.append((X_cat_b_t,y_cat_b_t))
    
  onlyOnce = False
  local_Xy_train_batch.append(local_Xy_train)
  n_pts_training += 260

##averaging of weights at central

In [ ]:
# weights_array = [[0],[1],[2],.....[9]]
# weights_array[i] = [[0],[1],[2],.....[15]]
# weights_array[i][j] is a np array

def federated_averaging(weights_array):
  mean_weights = []

  for i in range(len(weights_array[0])):    # i = [0,..,15]
    flag = True
    for j in range(len(weights_array)):     # j = [0,..,9]
      if(flag):
        temp = np.full(weights_array[j][i].shape, 0, dtype=np.float32) # temp full of 0(zeros)
        temp = weights_array[j][i] # this step is important bcoz 0+x/2 will result in error
        flag = False
      
      temp = np.mean((temp,weights_array[j][i]), axis=0) 
    mean_weights.append(temp)

  updated_model = define_model()
  updated_model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),loss='categorical_crossentropy', metrics=['accuracy'])
  updated_model.set_weights(mean_weights)
  _, accuracy = updated_model.evaluate(testX, testY,verbose=0) # returns loss and accuracy
  return mean_weights, accuracy


#GMM

In [ ]:
# from sklearn.mixture import GaussianMixture
# from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
# # finding optimal number of cluster in the local data
# # The number of clusters for each of the dataset in not known (unsupervised)

def minSilhouetteValue(local_X, local_y):
  gmm_list = []
  silhouette_val = []
  for n_comp in range(2,8): # n_comp [2,3,4,5,6,7]
    gmm = GaussianMixture(n_components=n_comp, random_state=10)
    cluster_labels = gmm.fit_predict(local_X)
    silhouette_val.append(silhouette_score(local_X, cluster_labels))
    gmm_list.append(gmm)
  
  min_val_index = silhouette_val.index(min(silhouette_val))
  print(gmm_list[min_val_index])
  return gmm_list[min_val_index]


In [ ]:
# Send local GMMs to Server & Do sampling at Server

def syntheticDataGen(gmm_list):
  n_samp = 20 # hyperparameter 

  syn_X_list = []
  syn_y_list = []

  for i in range(10):
    gmm = gmm_list[i]
    syn_X1,_ = gmm.sample(n_samp)
    syn_y1 = np.full(n_samp,i)
    print (syn_X1.shape,syn_y1.shape)

    syn_X_list.append(syn_X1)
    syn_y_list.append(syn_y1)

  syn_X = np.concatenate(syn_X_list)
  syn_y = np.concatenate(syn_y_list)
  syn_y = to_categorical(syn_y)
  print (syn_X, syn_y)
  return syn_X, syn_y  

In [ ]:
# def updateCentralModelGMM(syn_X, syn_y,X_central_train,y_central_train):
#   updated_model = define_model()
#   updated_model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),loss='categorical_crossentropy', metrics=['accuracy'])

#   # for i in range(len(syn_X)):
#   X_central_train = np.concatenate((X_central_train,syn_X.reshape(200,32,32,3)),axis=0)
  
#   y_central_train = np.concatenate((y_central_train,syn_y),axis=0)
#   print (X_central_train.shape,y_central_train.shape)

#   updated_model.fit(X_central_train, y_central_train, epochs=20, batch_size=100, validation_data=(testX, testY))

#   _, accuracy = updated_model.evaluate(testX, testY,verbose=0) # returns loss and accuracy

#   return accuracy


#Creating Local Model

#Iteration [1-10]

In [ ]:
for i in range(10):
  print("Cycle : ", i)
  weights_local = []

  local_model = define_model()
  local_model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),loss='categorical_crossentropy', metrics=['accuracy'])

  for j in range(1):
    # local model for Jth local client
    local_model.set_weights(weights_central)

    # local dataset for cycle i & jth client
    X_l_train, y_l_train = local_Xy_train_batch[i][j]

    local_model.fit(X_l_train, y_l_train, epochs=10, batch_size=100, validation_data=(testX, testY),verbose=1)
    localweights = local_model.get_weights()
    weights_local.append(localweights)

  # averaging of weights function call
  mean_weights_each_cycle, new_acc_avg = federated_averaging(weights_local)
  accuracy_central_Avg.append(new_acc_avg)
  weights_central = mean_weights_each_cycle

  

In [ ]:
# for i in range(10):
#   print("Cycle : ", i)
#   gmm_list = []

#   for j in range(10):

#     # local dataset for cycle i & jth client
#     X_l_train, y_l_train = local_Xy_train_batch[i][j]

#     # n_comp = minSilhouetteValue(X_l_train.reshape(-1,1), y_l_train)
#     # print(n_comp)

#     gmm_i = GaussianMixture(n_components=3)
#     gmm_i.fit(X = X_l_train.reshape(-1,3072))
#     gmm_list.append(gmm_i)

#   # synthetic/proxy data generation
#   syn_X1, syn_y1 = syntheticDataGen(gmm_list)
#   print(syn_X1.shape, X_central_train.shape)
#   print(syn_X1.shape, X_central_train.shape)

#   new_acc_gmm = updateCentralModelGMM(syn_X1, syn_y1, X_central_train, y_central_train)
#   accuracy_central_GMM.append(new_acc_gmm)

  

In [ ]:
for i in range(10):

  print("Cycle : ", i)
  weights_local = []
  gmm_list = []

  local_model = define_model()
  local_model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),loss='categorical_crossentropy', metrics=['accuracy'])

  for j in range(10):
    # local model for Jth local client
    local_model.set_weights(weights_central)

    # local dataset for cycle i & jth client
    X_l_train, y_l_train = local_Xy_train_batch[i][j]

    local_model.fit(X_l_train, y_l_train, epochs=10, batch_size=100, validation_data=(testX, testY),verbose=1)
    localweights = local_model.get_weights()
    weights_local.append(localweights)

    # n_comp = minSilhouetteValue(X_l_train.reshape(-1,1), y_l_train)
    # print(n_comp)

    gmm_i = GaussianMixture(n_components=3)
    gmm_i.fit(X = X_l_train.reshape(-1,3072))
    gmm_list.append(gmm_i)

  # averaging of weights function call
  mean_weights_each_cycle, new_acc_avg = federated_averaging(weights_local)
  accuracy_central_Avg.append(new_acc_avg)
  weights_central = mean_weights_each_cycle

  # synthetic/proxy data generation
  syn_X1, syn_y1 = syntheticDataGen(gmm_list)
  print(syn_X1.shape, X_central_train.shape)
  print(syn_X1.shape, X_central_train.shape)

  new_acc_gmm = updateCentralModelGMM(syn_X1, syn_y1, X_central_train, y_central_train)
  accuracy_central_GMM.append(new_acc_gmm)

  

In [ ]:
from prettytable import PrettyTable 

def printAccuracy(accuracy):
  accTable1 = PrettyTable(["Cycle ", "Accuracy"])

  for i in range(10):
    accTable1.add_row([str(i),str(accuracy[i])])
  print('\n')
  print(accTable1)

In [ ]:
# Print Accuracy
print("Averaging")
printAccuracy(accuracy_central_Avg)

# print("GMM")
# printAccuracy(accuracy_central_GMM)

Averaging


+--------+---------------------+
| Cycle  |       Accuracy      |
+--------+---------------------+
|   0    |         0.56        |
|   1    | 0.26179999113082886 |
|   2    | 0.33959999680519104 |
|   3    |  0.3301999866962433 |
|   4    |  0.3695000112056732 |
|   5    | 0.34279999136924744 |
|   6    |  0.335999995470047  |
|   7    | 0.37220001220703125 |
|   8    | 0.37619999051094055 |
|   9    | 0.35179999470710754 |
+--------+---------------------+
